In [1]:
import pandas as pd
import json
import numpy as np
import os
import glob
import datetime

# read data

In [2]:
#read race data for 2016-2017 and 2017-2018
data = pd.read_csv('/Users/yilanchen/Desktop/case fall/cleaned_no_classification/contest data.csv',sep=';')

In [3]:
print(glob.glob("/Users/yilanchen/Desktop/case fall/*"))

['/Users/yilanchen/Desktop/case fall/2018-2019', '/Users/yilanchen/Desktop/case fall/Wheather', '/Users/yilanchen/Desktop/case fall/cleaned_no_classification', '/Users/yilanchen/Desktop/case fall/Test', '/Users/yilanchen/Desktop/case fall/2015-2016', '/Users/yilanchen/Desktop/case fall/2014-2015', '/Users/yilanchen/Desktop/case fall/Formula_E_Project (1).ipynb', '/Users/yilanchen/Desktop/case fall/2016-2017', '/Users/yilanchen/Desktop/case fall/2017-2018']


In [4]:
place_20162017=os.listdir('/Users/yilanchen/Desktop/case fall/2016-2017')
place_20172018=os.listdir('/Users/yilanchen/Desktop/case fall/2017-2018')

In [5]:
a=['/Users/yilanchen/Desktop/case fall/2016-2017','/Users/yilanchen/Desktop/case fall/2017-2018']

In [6]:
datafinal=pd.DataFrame()

In [7]:
#style: FP1 FP2 Q
for i in range(len(a)):
    place_name=os.listdir(a[i])
    oo=a[i]+'/*'
    place_path=glob.glob(oo)
    for j in range(len(place_path)):
        ooo=place_path[j]+'/*'
        style_name=os.listdir(place_path[j])
        style_path=glob.glob(ooo)
        for k in range(len(style_path)):
            oooo=style_path[k] + '/*'
            final_csv=glob.glob(oooo)
            for m in range(len(final_csv)):
                if 'Classification' in final_csv[m]:
                    pass
                else:
                    p=pd.read_csv(final_csv[m],encoding='latin-1', sep=';')
                    p['style']=style_name[k]
                    p['place']=place_name[j]
                    p['time']=a[i][-9:]

                    datafinal=pd.concat([datafinal,p],axis=0)

In [8]:
datafinal_weather=pd.DataFrame()

In [9]:
time_name=os.listdir('/Users/yilanchen/Desktop/case fall/Wheather/16-18')
oo='/Users/yilanchen/Desktop/case fall/Wheather/16-18'+'/*'
time_path=glob.glob(oo)
for j in range(len(time_path)):
    ooo=time_path[j]+'/*'
    place_name=os.listdir(time_path[j])
    place_path=glob.glob(ooo)
    for k in range(len(place_path)):
        oooo=place_path[k] + '/*'
        style_name=os.listdir(place_path[k])
        style_path=glob.glob(oooo)
        for n in range(len(style_path)):
            ooooo=style_path[n]+'/*'
            final_csv=glob.glob(ooooo)
            for m in range(len(final_csv)):
                p=pd.read_csv(final_csv[m],encoding='latin-1', sep=';')
                p['style']=style_name[n]
                p['place']=place_name[k]
                p['time']=time_name[j]
                datafinal_weather=pd.concat([datafinal_weather,p],axis=0)

# race data process

In [14]:
#modify the column names
datafinal.rename(columns={'ï»¿NUMBER':'Number'},inplace=True)
datafinal.columns=datafinal.columns.str.strip()

In [15]:
#drop NA, including.....
datafinal=datafinal.drop(columns=['CLASS','Unnamed: 25','TOP_SPEED','GROUP','MANUFACTURER','CROSSING_FINISH_LINE_IN_PIT'],axis=1)

In [16]:
datafinal=datafinal[datafinal['place']!='.DS_Store']
datafinal=datafinal[datafinal['style']!='.DS_Store']
datafinal=datafinal[datafinal['time']!='.DS_Store']

In [17]:
datafinal=datafinal.reset_index().drop('index',axis=1)
datafinal['DRIVER_NAME']=datafinal['DRIVER_NAME'].map(lambda x:x.upper())

In [20]:
#get the key, using driver name, number, place and time(season year)
key_list=[]
for i in range(len(datafinal)):
    key_list.append(str(datafinal['Number'][i])+datafinal['DRIVER_NAME'][i]+str(datafinal['style'][i])+str(datafinal['place'][i])+str(datafinal['time'][i]))
datafinal['keys'] = key_list

In [28]:
#take 'max' as the maximum lap that each driver achieved in each game
group_datafinal=datafinal.groupby(['Number','DRIVER_NAME','style','place','time']).LAP_NUMBER.agg(['max'])
group_datafinal.head(10)

max
Number DRIVER_NAME     style place       time          
1      LUCAS DI GRASSI FP1   Berlin      2017-2018   21
                             Hong Kong   2017-2018   26
                             Hong Kong2  2017-2018   29
                             Marrakesh   2017-2018   25
                             Mexico city 2017-2018   28
                             Newyork     2017-2018   22
                             Newyork2    2017-2018   23
                             Paris       2017-2018   28
                             Punta       2017-2018   26
                             Rome        2017-2018   17

In [29]:
#reset the index
group_datafinal=group_datafinal.reset_index()
group_datafinal['kk']=group_datafinal['Number'].astype(str)+group_datafinal['DRIVER_NAME']+group_datafinal['style']+group_datafinal['place']+group_datafinal['time']

In [33]:
group_datafinal.head()

,Number,DRIVER_NAME,style,place,time,max,kk
0,1,LUCAS DI GRASSI,FP1,Berlin,2017-2018,21,1LUCAS DI GRASSIFP1Berlin2017-2018
1,1,LUCAS DI GRASSI,FP1,Hong Kong,2017-2018,26,1LUCAS DI GRASSIFP1Hong Kong2017-2018
2,1,LUCAS DI GRASSI,FP1,Hong Kong2,2017-2018,29,1LUCAS DI GRASSIFP1Hong Kong22017-2018
3,1,LUCAS DI GRASSI,FP1,Marrakesh,2017-2018,25,1LUCAS DI GRASSIFP1Marrakesh2017-2018
4,1,LUCAS DI GRASSI,FP1,Mexico city,2017-2018,28,1LUCAS DI GRASSIFP1Mexico city2017-2018


In [34]:
lap_dict = dict(zip(group_datafinal['kk'], group_datafinal['max']))

In [38]:
datafinal['max_lap']=datafinal['keys'].map(lap_dict)
datafinal['max_lap'].unique()

array([38, 49, 47, 33, 18, 48, 35,  3, 29, 30, 27, 31, 23, 26, 28, 24, 22,
       17, 13, 20, 15, 16, 11, 14, 19, 21, 43, 42,  2, 25, 32,  5, 46, 12,
       10,  7,  8,  9, 37, 34, 36,  4, 45, 44,  6, 51, 50, 40,  1, 39])

In [36]:
#set our target to be the remaining laps
datafinal['target'] = datafinal['max_lap']-datafinal['LAP_NUMBER']

### weather

In [40]:
#cleaning the data
datafinal_weather=datafinal_weather.reset_index().drop('index',axis=1)
for i in ['AIR_TEMP', 'TRACK_TEMP', 'PRESSURE', 'WIND_SPEED']:
    datafinal_weather[i]=datafinal_weather[i].map(lambda x:float(str(x).replace(',','.')))

In [41]:
#aggregate mean, std, min, max and median for air_temp, track_temp, humidity, pressure and wind_speed
datafinal_weather['match_key'] = datafinal_weather['style']+datafinal_weather['place']+datafinal_weather['time']
datafinal_weather=datafinal_weather.groupby('match_key',as_index=True).agg(
            mean_air_temp=('AIR_TEMP', 'mean'),
             std_air_temp=('AIR_TEMP', 'std'),
             max_air_temp=('AIR_TEMP', 'max'),
             min_air_temp=('AIR_TEMP', 'min'),
             med_air_temp=('AIR_TEMP', 'median'),
            mean_track_temp=('TRACK_TEMP', 'mean'),
             std_track_temp=('TRACK_TEMP', 'std'),
             max_track_temp=('TRACK_TEMP', 'max'),
             min_track_temp=('TRACK_TEMP', 'min'),
             med_track_temp=('TRACK_TEMP', 'median'),
            mean_humidity=('HUMIDITY', 'mean'),
             std_humidity=('HUMIDITY', 'std'),
             max_humidity=('HUMIDITY', 'max'),
             min_humidity=('HUMIDITY', 'min'),
             med_humidity=('HUMIDITY', 'median'),
            mean_pressure=('PRESSURE', 'mean'),
             std_pressure=('PRESSURE', 'std'),
             max_pressure=('PRESSURE', 'max'),
             min_pressure=('PRESSURE', 'min'),
             med_pressure=('PRESSURE', 'median'),
            mean_wind_speed=('WIND_SPEED', 'mean'),
             std_wind_speed=('WIND_SPEED', 'std'),
             max_wind_speed=('WIND_SPEED', 'max'),
             min_wind_speed=('WIND_SPEED', 'min'),
             med_wind_speed=('WIND_SPEED', 'median')).reset_index()

# match data

In [57]:
#set match key: same season year, same 'style' and same location. in this case, the driver name and their teams does not matter
datafinal['match_key']=datafinal['style']+datafinal['place']+datafinal['time']
match_data = pd.merge(datafinal,datafinal_weather,on='match_key')

In [58]:
match_data = match_data.drop(['keys','time','style','max_lap','DRIVER_NAME','HOUR','DRIVER_NUMBER','Number','place','ELAPSED','S1_LARGE'
                             ,'S2_LARGE','S3_LARGE','PIT_TIME','match_key','TEAM'],axis=1)

In [59]:
#process the lap time 
def process_laptime(time):
    if ':' in time:
        try:
            return (datetime.datetime.strptime(time, "%M:%S.%f")-datetime.datetime(1900,1,1)).total_seconds()
        except:
            return (datetime.datetime.strptime(time, "%H:%M:%S.%f")-datetime.datetime(1900,1,1)).total_seconds()
    else:
        return (datetime.datetime.strptime(time, "%S.%f")-datetime.datetime(1900,1,1)).total_seconds()
match_data['LAP_TIME']= match_data['LAP_TIME'].map(process_laptime)

In [53]:
match_data.head()

,LAP_NUMBER,LAP_TIME,LAP_IMPROVEMENT,S1,S1_IMPROVEMENT,S2,S2_IMPROVEMENT,S3,S3_IMPROVEMENT,KPH,...,mean_pressure,std_pressure,max_pressure,min_pressure,med_pressure,mean_wind_speed,std_wind_speed,max_wind_speed,min_wind_speed,med_wind_speed
0,1,66.919,0,12.766,0,31.602,0,22.551,0,103.3,...,1016.751786,0.052427,1016.83,1016.63,1016.77,4.080836,1.883035,8.04672,1.60934,4.02336
1,2,67.695,0,18.342,0,27.325,0,22.028,0,102.1,...,1016.751786,0.052427,1016.83,1016.63,1016.77,4.080836,1.883035,8.04672,1.60934,4.02336
2,3,66.203,0,18.153,0,26.069,0,21.981,0,104.4,...,1016.751786,0.052427,1016.83,1016.63,1016.77,4.080836,1.883035,8.04672,1.60934,4.02336
3,4,65.568,0,18.224,0,25.7,0,21.644,0,105.4,...,1016.751786,0.052427,1016.83,1016.63,1016.77,4.080836,1.883035,8.04672,1.60934,4.02336
4,5,65.341,0,18.162,0,25.621,0,21.558,0,105.8,...,1016.751786,0.052427,1016.83,1016.63,1016.77,4.080836,1.883035,8.04672,1.60934,4.02336


In [62]:
match_data.to_csv('/Users/yilanchen/Desktop/case fall/match_data/match_data.csv')